# Módulo 2.1: CNN para Clasificación de Imágenes con MLflow

## Teoría: Redes Neuronales Convolucionales (CNN)

### ¿Qué es una CNN?
Las Redes Neuronales Convolucionales son arquitecturas de Deep Learning especializadas en procesar datos con estructura de cuadrícula (imágenes, video, señales).

### Componentes principales:

1. **Capas Convolucionales (Conv2D)**
   - Aplican filtros/kernels para detectar patrones
   - Aprenden características jerárquicas
   - Comparten pesos (parameter sharing)
   - Invarianza translacional

2. **Capas de Pooling**
   - MaxPooling: toma el valor máximo
   - AveragePooling: calcula promedio
   - Reduce dimensionalidad
   - Aporta invarianza a pequeñas translaciones

3. **Capas Densas (Fully Connected)**
   - Combinan features aprendidas
   - Producen predicción final

4. **Funciones de Activación**
   - **ReLU**: $f(x) = max(0, x)$ - más común en capas intermedias
   - **Softmax**: Para clasificación multiclase
   - **Sigmoid**: Para clasificación binaria

5. **Regularización**
   - **Dropout**: Apaga neuronas aleatoriamente durante entrenamiento
   - **Batch Normalization**: Normaliza activaciones
   - **Data Augmentation**: Aumenta variedad de datos

### Arquitectura típica:
```
Input Image
    ↓
[Conv2D + ReLU + Pooling] × N
    ↓
Flatten
    ↓
[Dense + ReLU + Dropout] × M
    ↓
Dense + Softmax
    ↓
Output (Predictions)
```

## Objetivos
- Construir CNNs con TensorFlow/Keras
- Integración completa con MLflow
- Tracking de arquitecturas DL
- Callbacks personalizados
- Logging de métricas por época
- Visualización de training history

In [ ]:
import mlflow
import mlflow.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.datasets import mnist, fashion_mnist, cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")
print(f"MLflow version: {mlflow.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("tensorflow-cnn-classification")

## 1. Dataset: MNIST (Dígitos escritos a mano)

Dataset clásico de 70,000 imágenes de dígitos (0-9) de 28x28 píxeles

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(f"Train images: {X_train.shape}")
print(f"Train labels: {y_train.shape}")
print(f"Test images: {X_test.shape}")
print(f"Test labels: {y_test.shape}")

print(f"\nClases: {np.unique(y_train)}")
print(f"Rango de píxeles: [{X_train.min()}, {X_train.max()}]")

### Visualización de ejemplos

In [ ]:
fig, axes = plt.subplots(3, 10, figsize=(15, 5))
for i in range(30):
    ax = axes[i // 10, i % 10]
    ax.imshow(X_train[i], cmap='gray')
    ax.set_title(f'Label: {y_train[i]}')
    ax.axis('off')
plt.tight_layout()
plt.savefig('mnist_samples.png', dpi=150)
plt.show()

### Preprocesamiento

In [ ]:
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

print(f"Train images normalized: {X_train.shape}")
print(f"Train labels one-hot: {y_train_cat.shape}")
print(f"Pixel range after normalization: [{X_train.min():.2f}, {X_train.max():.2f}]")

## 2. Modelo Simple: CNN Básica

Arquitectura básica:
- Conv2D(32 filters) → MaxPooling
- Conv2D(64 filters) → MaxPooling
- Flatten
- Dense(128) → Dropout
- Dense(10, softmax)

In [ ]:
def create_simple_cnn(input_shape=(28, 28, 1), num_classes=10):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

model = create_simple_cnn()
model.summary()

## 3. Custom MLflow Callback

Callback personalizado para logging automático de métricas en cada época

In [ ]:
class MLflowCallback(callbacks.Callback):
    """
    Callback personalizado para logging de métricas en MLflow
    """
    def __init__(self, log_every_n_epochs=1):
        super().__init__()
        self.log_every_n_epochs = log_every_n_epochs
    
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.log_every_n_epochs == 0:
            logs = logs or {}
            
            for metric_name, metric_value in logs.items():
                mlflow.log_metric(metric_name, metric_value, step=epoch)
            
            print(f"Epoch {epoch + 1}: Métricas logged to MLflow")

## 4. Entrenamiento con MLflow Tracking

In [ ]:
with mlflow.start_run(run_name="simple_cnn_mnist") as run:
    
    model = create_simple_cnn()
    
    optimizer = 'adam'
    loss = 'categorical_crossentropy'
    batch_size = 128
    epochs = 10
    
    mlflow.log_param("model_type", "SimpleCNN")
    mlflow.log_param("optimizer", optimizer)
    mlflow.log_param("loss", loss)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("input_shape", "28x28x1")
    
    total_params = model.count_params()
    mlflow.log_param("total_parameters", total_params)
    
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
    
    early_stopping = callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    
    mlflow_callback = MLflowCallback(log_every_n_epochs=1)
    
    history = model.fit(
        X_train, y_train_cat,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.1,
        callbacks=[early_stopping, mlflow_callback],
        verbose=1
    )
    
    test_loss, test_accuracy = model.evaluate(X_test, y_test_cat, verbose=0)
    
    mlflow.log_metric("final_test_loss", test_loss)
    mlflow.log_metric("final_test_accuracy", test_accuracy)
    
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    cm = confusion_matrix(y_test, y_pred_classes)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix - Simple CNN')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig('confusion_matrix_simple_cnn.png')
    mlflow.log_artifact('confusion_matrix_simple_cnn.png')
    plt.close()
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    axes[0].plot(history.history['loss'], label='Train Loss')
    axes[0].plot(history.history['val_loss'], label='Val Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training and Validation Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(history.history['accuracy'], label='Train Accuracy')
    axes[1].plot(history.history['val_accuracy'], label='Val Accuracy')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('training_history.png', dpi=150)
    mlflow.log_artifact('training_history.png')
    plt.show()
    
    mlflow.keras.log_model(model, "cnn_model")
    
    mlflow.set_tag("dataset", "MNIST")
    mlflow.set_tag("framework", "TensorFlow/Keras")
    mlflow.set_tag("architecture", "CNN")
    
    print(f"\nRun ID: {run.info.run_id}")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Total Parameters: {total_params:,}")

## 5. Modelo Avanzado: CNN con Batch Normalization

Mejoras:
- Batch Normalization después de cada Conv2D
- Más capas convolucionales
- Dropout adicional

In [ ]:
def create_advanced_cnn(input_shape=(28, 28, 1), num_classes=10):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

model_adv = create_advanced_cnn()
model_adv.summary()

In [ ]:
with mlflow.start_run(run_name="advanced_cnn_mnist") as run:
    
    model = create_advanced_cnn()
    
    lr = 0.001
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    batch_size = 128
    epochs = 15
    
    mlflow.log_param("model_type", "AdvancedCNN")
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("learning_rate", lr)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("batch_normalization", True)
    mlflow.log_param("total_parameters", model.count_params())
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    reduce_lr = callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-6,
        verbose=1
    )
    
    early_stopping = callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )
    
    mlflow_callback = MLflowCallback()
    
    history = model.fit(
        X_train, y_train_cat,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.1,
        callbacks=[early_stopping, reduce_lr, mlflow_callback],
        verbose=1
    )
    
    test_loss, test_accuracy = model.evaluate(X_test, y_test_cat, verbose=0)
    
    mlflow.log_metric("final_test_loss", test_loss)
    mlflow.log_metric("final_test_accuracy", test_accuracy)
    
    mlflow.keras.log_model(model, "advanced_cnn_model")
    
    mlflow.set_tag("dataset", "MNIST")
    mlflow.set_tag("architecture", "AdvancedCNN")
    
    print(f"\nTest Accuracy: {test_accuracy:.4f}")
    print(f"Improvement over simple CNN: {(test_accuracy - 0.99)*100:.2f}%")

## 6. Data Augmentation

### Teoría: Data Augmentation
Técnica para aumentar artificialmente el tamaño del dataset aplicando transformaciones:
- Rotaciones
- Desplazamientos (shifts)
- Zoom
- Flips horizontales/verticales
- Cambios de brillo

**Beneficios**:
- Reduce overfitting
- Mejora generalización
- Hace el modelo más robusto

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)

sample_image = X_train[0:1]
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.ravel()

for i, ax in enumerate(axes):
    if i == 0:
        ax.imshow(sample_image[0, :, :, 0], cmap='gray')
        ax.set_title('Original')
    else:
        augmented = datagen.flow(sample_image, batch_size=1)
        img = next(augmented)[0]
        ax.imshow(img[:, :, 0], cmap='gray')
        ax.set_title(f'Augmented {i}')
    ax.axis('off')

plt.tight_layout()
plt.savefig('data_augmentation_examples.png', dpi=150)
plt.show()

## 7. Comparación de Modelos

In [ ]:
experiment = mlflow.get_experiment_by_name("tensorflow-cnn-classification")
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])

print("Comparación de modelos CNN:")
comparison = runs[[
    'tags.mlflow.runName',
    'metrics.final_test_accuracy',
    'params.total_parameters',
    'params.batch_size',
    'params.epochs'
]].sort_values('metrics.final_test_accuracy', ascending=False)

print(comparison)

## Resumen del Módulo 2.1

### Conceptos Clave:

1. **CNNs para Visión por Computadora**
   - Capas convolucionales detectan patrones
   - Pooling reduce dimensionalidad
   - Arquitectura jerárquica

2. **Técnicas de Regularización**
   - Dropout: previene overfitting
   - Batch Normalization: estabiliza entrenamiento
   - Data Augmentation: aumenta datos

3. **MLflow con TensorFlow/Keras**
   - `mlflow.keras.log_model()`: guardar modelos
   - Custom callbacks para logging automático
   - Tracking de métricas por época
   - Logging de arquitectura y parámetros

4. **Callbacks de Keras**
   - EarlyStopping: detiene cuando no mejora
   - ReduceLROnPlateau: ajusta learning rate
   - Custom MLflowCallback: logging automático

### Mejores Prácticas:
- Normalizar imágenes (0-1)
- Usar validation split
- Aplicar data augmentation
- Monitorear overfitting
- Guardar mejores pesos

### Próximo Notebook:
RNN/LSTM para series temporales